# Scopus Scrapper

In [23]:
queryText = "a"
counts = 2

# Your API key obtained from Elsevier Developer Portal
API_KEY = 'ba849e4aa45cb76492990d7715355f7e'

In [24]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np

## Get all researches, authors, and affiliations id from provided data

In [30]:
researches_id_set = set()
authors_id_set = set()
affiliations_id_set = set()

In [33]:
provided_researches_df = pd.read_csv("../data/researches.csv", index_col=0)
provided_authors_df = pd.read_csv("../data/authors.csv", index_col=0)
provided_affiliations_df = pd.read_csv("../data/affiliations.csv", index_col=0)

In [34]:
researches_id_set = set(provided_researches_df['id'])
authors_id_set = set(provided_authors_df['id'])
affiliations_id_set = set(provided_affiliations_df['id'])

## Scraping from Scopus API

In [17]:
# Scopus API endpoint for document search
SCOPUS_API_URL = 'https://api.elsevier.com/content/search/scopus'

def query_scopus(query, count=25):
    params = {
        'apiKey': API_KEY,  # Include your API key here
        'query': query,
        'count': count
    }

    response = requests.get(SCOPUS_API_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to query Scopus API. Status code: {response.status_code}")
        return None

In [18]:
listDOI = list()
listID = list()

def scopusFinder(queryText , counts ):

    query = queryText
    count = counts

    results = query_scopus(query, count)
    if results:
        print("Search Successful")
    else:
        print("Search Failed")
        # for paper in results['search-results']['entry']:
        #     print(paper)
        #     print("-" * 50)
        #     print("Title:", paper['dc:title'])

        #     # Check if dc:creator is a list of dictionaries
        #     creators = paper.get('dc:creator', [])
        #     if isinstance(creators, list):
        #         print("Authors:", ', '.join(author.get('surname', '') + ' ' + author.get('given-name', '') for author in creators))
        #     else:
        #         print("Authors:", creators)

        #     print("Publication Year:", paper['prism:coverDate'].split('-')[0])
        #     print("DOI:", paper['prism:doi'])
        #     print("Identifier:", paper['dc:identifier'])
        #     listDOI.append(paper['prism:doi'])
        #     listID.append(paper['dc:identifier'].split(":")[1])
        #     print("Abstract:", paper.get('dc:description', 'N/A'))  # Extract and print abstract
        #     print("Keywords:", ', '.join(keyword['$'] for keyword in paper.get('authkeywords', [])))

        #     # Placeholder for retrieving and printing full-text content
        #     print("Full Text: [Placeholder for full-text content retrieval]")

        #     print("-" * 50)

In [19]:
def get_full_text_id(id):
    crossref_api_url = f'https://api.elsevier.com/content/abstract/scopus_id/{id}'
    response = requests.get(crossref_api_url ,headers={"Accept" : "application/json","X-ELS-APIKey" : API_KEY },params={"view" : "FULL"})
    if response.status_code == 200:
        data = json.loads(response.text)
        return data
    else:
        print(f"Failed to fetch full-text. Status code: {response.status_code}")
    return None

In [20]:
listID = list()
scopusFinder(queryText , counts )

Failed to query Scopus API. Status code: 400
Search Failed


In [9]:
listID

['85191295877', '85191240117']

In [10]:
dic = {}

In [11]:
listdoc = list()
i = 1

n = len(listID)
while(True):
  i = 1
  sidi = 0
  while sidi < n:
    sid = listID[sidi]
    print(sid)
    data = get_full_text_id(sid)
    sidi += 1
    print(data)
    if data:
      dic[i] = {}
      dic[i]["citation-title"] =  data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-title']
      dic[i]['abstracts'] = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
      i+=1
    # sidi += 1
  if(sidi == n):
    break

85191295877
{'abstracts-retrieval-response': {'item': {'ait:process-info': {'ait:status': {'@state': 'new', '@type': 'core', '@stage': 'S300'}, 'ait:date-delivered': {'@day': '01', '@timestamp': '2024-05-01T08:46:56.000056-04:00', '@year': '2024', '@month': '05'}, 'ait:date-sort': {'@day': '01', '@year': '2025', '@month': '12'}}, 'bibrecord': {'head': {'author-group': {'affiliation': {'country': 'India', 'address-part': 'Andhra Pradesh', '@afid': '60119614', '@country': 'ind', 'city': 'Amaravathi', 'organization': [{'$': 'School of Computer Science and Engineering'}, {'$': 'VIT-AP University'}], 'affiliation-id': {'@afid': '60119614', '@dptid': '123061224'}, '@affiliation-instance-id': 'OB2BibRecID-955907120-f527085ab8ba5bdd2992189ed4d1b71b-1', 'ce:source-text': 'School of Computer Science and Engineering, VIT-AP University, Amaravathi. Andhra Pradesh, India', '@dptid': '123061224'}, 'author': [{'preferred-name': {'ce:given-name': 'M.', 'ce:initials': 'M.', 'ce:surname': 'Kokila', 'ce:

In [13]:
data['abstracts-retrieval-response']['item']['bibrecord']['head'].keys()

dict_keys(['author-group', 'citation-title', 'abstracts', 'correspondence', 'citation-info', 'source', 'enhancement', 'grantlist'])

In [14]:
data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-title']

'Authentication, access control and scalability models in Internet of Things Security–A review'

In [15]:
data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']

'© 2024The Internet of Things (IoT) leads to the next phase of human interaction with technology. With the help of the IoT, physical objects can be given the ability to generate, receive, and seamlessly trade data with one another. The IoT includes a wide variety of applications, each of which focuses on automating a specific task and works to give inanimate objects the ability to act independently of human intervention. The currently available and upcoming IoT applications hold a great deal of promise for enhancing the level of convenience, productivity, and automation enjoyed by users. High levels of security, privacy, authentication, and the ability to recover from attacks are required for the implementation of such a world in a manner that is constantly expanding. In this light, it is necessary to make the necessary adjustments to the architecture of IoT applications to accomplish end-to-end security in IoT environments. In this article, a comprehensive review of the security-relat

In [ ]:
data